# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [146]:
import numpy as np
import pandas as pd

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [147]:
admissions = pd.read_csv('../data/Admission_Predict.csv')

Remove trailing spaces at the end of the column names if there are any.


In [148]:
admissions.columns = [col.strip() for col in admissions.columns]
admissions.columns

# or:
# admissions.columns = admissions.columns.str.rstrip()

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [149]:
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [150]:
admissions.isna().any()

Serial No.           False
GRE Score            False
TOEFL Score          False
University Rating    False
SOP                  False
LOR                  False
CGPA                 False
Research             False
Chance of Admit      False
dtype: bool

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [151]:
admissions.set_index('Serial No.', drop=False, inplace=True)
admissions.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,,
1,1,337,118,4,4.5,4.5,9.65,1,0.92
2,2,316,104,3,3.0,3.5,8.00,1,0.72
3,3,322,110,3,3.5,2.5,8.67,1,0.80
4,4,314,103,2,2.0,3.0,8.21,0,0.65
5,5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [152]:
len(admissions)

385

In [153]:
len(admissions['GRE Score'].unique())

49

In [154]:
len(admissions['CGPA'].unique())

168

In [155]:
# That is wrong, GRE Score has 49 unique values, and CGPA has 168. The dataframe has 385 rows so neither of them are unique identifiers.

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [156]:
# Set the multi-index using 'GRE Score' and 'CGPA'
admissions.set_index(['GRE Score', 'CGPA'], inplace=True)

In [157]:
# Show the updated DataFrame
admissions.head()

,,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,,
337,9.65,1,118,4,4.5,4.5,1,0.92
316,8.00,2,104,3,3.0,3.5,1,0.72
322,8.67,3,110,3,3.5,2.5,1,0.80
314,8.21,4,103,2,2.0,3.0,0,0.65
330,9.34,5,115,5,4.5,3.0,1,0.90


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [158]:
admissions.reset_index(inplace=True)
admissions.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,1,118,4,4.5,4.5,1,0.92
1,316,8.00,2,104,3,3.0,3.5,1,0.72
2,322,8.67,3,110,3,3.5,2.5,1,0.80
3,314,8.21,4,103,2,2.0,3.0,0,0.65
4,330,9.34,5,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [159]:
len(admissions[(admissions['CGPA'] > 9) & (admissions['Research'] == 1)])

101

Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [160]:
filtered_data = admissions[(admissions['CGPA'] > 9) & (admissions['SOP'] < 3.5)]
mean_chance_admit = filtered_data['Chance of Admit'].mean()
print(mean_chance_admit)

0.8019999999999999


We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [161]:
def standardize_column(column):
    mean = np.mean(column)
    std_dev = np.std(column)
    standardized_column = (column - mean) / std_dev
    return standardized_column

In [162]:
# Standardize the columns and assign them to new columns
admissions['CGPA_std'] = standardize_column(admissions['CGPA'])
admissions['GRE_std'] = standardize_column(admissions['GRE Score'])
admissions['LOR_std'] = standardize_column(admissions['LOR'])

We will generate the decision choice at random using the code below. Please run the cell.

In [163]:
# Libraries
from random import choices

In [164]:
# Run this code:

std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=admissions.shape[0])
decision_choice

['CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'LOR_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'GRE_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'GRE_std',
 'CGPA_std',
 'LOR_std',
 'GRE_std',
 'LOR_std',
 'CGPA_std',
 'CGPA_std',
 'GRE_std',
 

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [165]:
# There is no more lookup-function
admissions['deciding_column'] = admissions.apply(lambda row: row[decision_choice[row.name]], axis=1)
admissions.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [166]:
admissions['decision'] = admissions['deciding_column'].apply(lambda x: 1 if x > 0.8 else 0)
admissions.head()

,GRE Score,CGPA,Serial No.,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [167]:
print(admissions['decision'].sum())

84


# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [168]:
admissions.columns = (admissions.columns
                      .str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)  # Remove non-character symbols
                      .str.replace(' ', '_')  # Replace spaces with underscores
                      .str.lower())  # Convert to lowercase

admissions.head()

,gre_score,cgpa,serial_no,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpastd,grestd,lorstd,decidingcolumn,decision
0,337,9.65,1,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.750174,1
1,316,8.00,2,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,3,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,4,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.236698,0
4,330,9.34,5,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [169]:
# Apply the 10-point boost for students with a university rating of 4 or higher
admissions['adjusted_gre'] = admissions.apply(lambda row: row['gre_score'] + 10 if row['university_rating'] >= 4 else row['gre_score'], axis=1)

# Split the adjusted GRE scores into 4 bins
bins = [0, 300, 500, 700, 800]  # You can adjust the bin ranges if needed
labels = ['low', 'medium_low', 'medium_high', 'high']

# Create the new 'adjusted_gre' based on these bins
admissions['adjusted_gre_bins'] = pd.cut(admissions['adjusted_gre'], bins=bins, labels=labels, right=False)

# Display the updated DataFrame with the 'adjusted_gre' and 'adjusted_gre_bins' columns
admissions[['gre_score', 'university_rating', 'adjusted_gre', 'adjusted_gre_bins']].head()

,gre_score,university_rating,adjusted_gre,adjusted_gre_bins
0,337,4,347.0,medium_low
1,316,3,316.0,medium_low
2,322,3,322.0,medium_low
3,314,2,314.0,medium_low
4,330,5,340.0,medium_low
